In [11]:
!pip install -q -r requirements.txt


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python -m pip install --upgrade pip


In [12]:
import os
from pathlib import Path
from getpass import getpass
from datasets import load_dataset

import pandas as pd
import shutil
from tqdm.auto import tqdm
import ast
import pybboxes as pbx

import ultralytics
from ultralytics import YOLO
import wandb

import random

In [3]:
dataset = load_dataset("keremberke/blood-cell-object-detection", "full")

In [7]:
dataset_dir = "/workspace/object-detection-balloons/datasets/bloodcells"
os.makedirs(dataset_dir, exist_ok=True)

In [8]:
# rm -rf /workspace/object-detection-balloons/datasets/bloodcells

In [9]:
for split in dataset:
    for row in dataset[split]:
        img_path = Path(os.path.join(dataset_dir, "images", split, str(row["image_id"])) + ".jpg")
        os.makedirs(img_path.parent, exist_ok=True)
        row["image"].save(img_path)
        for bbox, category in zip(row["objects"]["bbox"], row["objects"]["category"]):
            bbox_yolo = pbx.convert_bbox(bbox, from_type="coco", to_type="yolo", image_size=(row["width"], row["height"]))
            bbox_yolo = " ".join([str(bb) for bb in bbox_yolo])
            file_path = Path(os.path.join(dataset_dir, "labels", split, str(row["image_id"])) + ".txt")
            os.makedirs(file_path.parent, exist_ok=True)
            with open(file_path, "a") as f:
                f.write(f"{category} {bbox_yolo}\n")

In [10]:
ultralytics.checks()

Ultralytics YOLOv8.0.168 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Setup complete ✅ (20 CPUs, 62.6 GB RAM, 1.2/20.0 GB disk)


In [6]:
val_img_base_path = "/workspace/object-detection-balloons/datasets/bloodcells/images/validation"
val_img_paths = [os.path.join(val_img_base_path, fname) for fname in os.listdir(val_img_base_path)]
print(val_img_paths[:3])

test_img_base_path = "/workspace/object-detection-balloons/datasets/bloodcells/images/test"
test_img_paths = [os.path.join(test_img_base_path, fname) for fname in os.listdir(test_img_base_path)]
print(test_img_paths[:3])

['/workspace/object-detection-balloons/datasets/bloodcells/images/validation/51.jpg', '/workspace/object-detection-balloons/datasets/bloodcells/images/validation/14.jpg', '/workspace/object-detection-balloons/datasets/bloodcells/images/validation/48.jpg']
['/workspace/object-detection-balloons/datasets/bloodcells/images/test/10.jpg', '/workspace/object-detection-balloons/datasets/bloodcells/images/test/32.jpg', '/workspace/object-detection-balloons/datasets/bloodcells/images/test/34.jpg']


## Baseline

In [4]:
checkpoint = 'yolov8m.pt'

In [7]:
yolo_base = YOLO(checkpoint)
preds = yolo_base.predict(val_img_paths, save=True, project="preds", name="baseline") 


0: 640x640 (no detections), 1: 640x640 1 donut, 2: 640x640 1 donut, 3: 640x640 1 cake, 4: 640x640 (no detections), 5: 640x640 (no detections), 6: 640x640 (no detections), 7: 640x640 1 donut, 1 teddy bear, 8: 640x640 (no detections), 9: 640x640 1 cake, 10: 640x640 1 person, 11: 640x640 (no detections), 12: 640x640 1 person, 13: 640x640 1 person, 14: 640x640 (no detections), 15: 640x640 1 teddy bear, 16: 640x640 (no detections), 17: 640x640 (no detections), 18: 640x640 1 cake, 19: 640x640 (no detections), 20: 640x640 1 donut, 21: 640x640 (no detections), 22: 640x640 (no detections), 23: 640x640 (no detections), 24: 640x640 (no detections), 25: 640x640 (no detections), 26: 640x640 1 orange, 27: 640x640 (no detections), 28: 640x640 (no detections), 29: 640x640 (no detections), 30: 640x640 1 donut, 31: 640x640 1 cake, 32: 640x640 (no detections), 33: 640x640 (no detections), 34: 640x640 (no detections), 35: 640x640 1 donut, 36: 640x640 (no detections), 37: 640x640 (no detections), 38: 640x

## Finetuning

In [14]:
wandb.init()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [19]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


lr/pg0,▁▄▆▇██▇▆▄▂
lr/pg1,▁▄▆▇██▇▆▄▂
lr/pg2,▁▄▆▇██▇▆▄▂
metrics/mAP50(B),▁▇▁▅██████
metrics/mAP50-95(B),▁▆▁▅▇▇▇███
metrics/precision(B),▆▅▁▄▆▇▇▆██
metrics/recall(B),▁▇▅▆█▇▇▇█▇
model/GFLOPs,▁
model/parameters,▁
model/speed(ms),▁
train/box_loss,█▄▂▂▂▂▁▁▁▁


In [16]:
from ultralytics.yolo.utils.torch_utils import get_flops, get_num_params

try:
    import wandb

    assert hasattr(wandb, '__version__')
except (ImportError, AssertionError):
    wandb = None


def on_pretrain_routine_start(trainer):
    wandb.init(project=trainer.args.project or "YOLOv8", 
               name=trainer.args.name, 
               save_code=True,
               config=dict(
        trainer.args)) if not wandb.run else wandb.run


def on_fit_epoch_end(trainer):
    wandb.run.log(trainer.metrics, step=trainer.epoch + 1)
    if trainer.epoch == 0:
        model_info = {
            "model/parameters": get_num_params(trainer.model),
            "model/GFLOPs": round(get_flops(trainer.model), 3),
            "model/speed(ms)": round(sum(trainer.validator.speed.values()), 3)}
        wandb.run.log(model_info, step=trainer.epoch + 1)


def on_train_epoch_end(trainer):
    wandb.run.log(trainer.label_loss_items(trainer.tloss, prefix="train"), step=trainer.epoch + 1)
    wandb.run.log(trainer.lr, step=trainer.epoch + 1)
    if trainer.epoch == 1:
        wandb.run.log({f.stem: wandb.Image(str(f))
                       for f in trainer.save_dir.glob('train_batch*.jpg')},
                      step=trainer.epoch + 1)


def on_train_end(trainer):
    art = wandb.Artifact(type="model", name=f"run_{wandb.run.id}_model")
    if trainer.best.exists():
        art.add_file(trainer.best)
        wandb.run.log_artifact(art)


callbacks = {
    "on_pretrain_routine_start": on_pretrain_routine_start,
    "on_train_epoch_end": on_train_epoch_end,
    "on_fit_epoch_end": on_fit_epoch_end,
    "on_train_end": on_train_end} if wandb else {}

WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")



In [20]:
yolo_finetuned = YOLO(checkpoint)  # load a pretrained model (recommended for training)
for event, func in callbacks.items():
    yolo_finetuned.add_callback(event, func)

dataset_yaml_path = "/workspace/object-detection-balloons/bloodcells.yaml"

In [21]:
# Use the model
results = yolo_finetuned.train(data=dataset_yaml_path, epochs=50, batch=32)  # train the model

Ultralytics YOLOv8.0.168 🚀 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/workspace/object-detection-balloons/bloodcells.yaml, epochs=50, patience=50, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True

  8                  -1  2   3985920  ultralytics.nn.modules.block.C2f             [576, 576, 2, True]           
  9                  -1  1    831168  ultralytics.nn.modules.block.SPPF            [576, 576, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 11             [-1, 6]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 12                  -1  2   1993728  ultralytics.nn.modules.block.C2f             [960, 384, 2]                 
 13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']          
 14             [-1, 4]  1         0  ultralytics.nn.modules.conv.Concat           [1]                           
 15                  -1  2    517632  ultralytics.nn.modules.block.C2f             [576, 192, 2]                 
 16                  -1  1    332160  ultralytics.nn.modules.conv.Conv             [192,

Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅
train: Scanning /workspace/object-detection-balloons/datasets/bloodcells/labels/train.cache... 255 images, 0 backgrounds, 0 corrupt: 100%|██████████| 255/255 [00:00<?, ?it/s]
val: Scanning /workspace/object-detection-balloons/datasets/bloodcells/labels/validation.cache... 73 images, 0 backgrounds, 0 corrupt: 100%|██████████| 73/73 [00:00<?, ?it/s]
Plotting labels to runs/detect/train7/labels.jpg... 
/usr/local/lib/python3.10/dist-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(vector):
/usr/local/lib/python3.10/dist-packages/seaborn/_oldcore.py:1498: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.

## Evaluation

### Validation Eval

In [9]:
!rm -rf /workspace/object-detection-balloons/preds/val
preds = yolo_finetuned.predict(val_img_paths, save=True, project="preds", name="val") 


0: 640x640 30 rbcs, 1 wbc, 1: 640x640 1 platelets, 15 rbcs, 1 wbc, 2: 640x640 2 plateletss, 29 rbcs, 1 wbc, 3: 640x640 1 platelets, 24 rbcs, 1 wbc, 4: 640x640 2 plateletss, 21 rbcs, 1 wbc, 5: 640x640 1 platelets, 29 rbcs, 1 wbc, 6: 640x640 15 rbcs, 1 wbc, 7: 640x640 2 plateletss, 14 rbcs, 1 wbc, 8: 640x640 17 rbcs, 1 wbc, 9: 640x640 3 plateletss, 23 rbcs, 1 wbc, 10: 640x640 1 platelets, 17 rbcs, 2 wbcs, 11: 640x640 1 platelets, 16 rbcs, 1 wbc, 12: 640x640 18 rbcs, 1 wbc, 13: 640x640 1 platelets, 20 rbcs, 1 wbc, 14: 640x640 5 plateletss, 23 rbcs, 1 wbc, 15: 640x640 1 platelets, 16 rbcs, 1 wbc, 16: 640x640 1 platelets, 18 rbcs, 1 wbc, 17: 640x640 1 platelets, 27 rbcs, 1 wbc, 18: 640x640 1 platelets, 27 rbcs, 1 wbc, 19: 640x640 1 platelets, 26 rbcs, 1 wbc, 20: 640x640 22 rbcs, 1 wbc, 21: 640x640 2 plateletss, 20 rbcs, 2 wbcs, 22: 640x640 23 rbcs, 1 wbc, 23: 640x640 23 rbcs, 1 wbc, 24: 640x640 1 platelets, 20 rbcs, 1 wbc, 25: 640x640 22 rbcs, 1 wbc, 26: 640x640 3 plateletss, 25 rbcs, 1 wb

### Test Eval

In [10]:
!rm -rf /workspace/object-detection-balloons/preds/test
preds = yolo_finetuned.predict(test_img_paths, save=True, project="preds", name="test") 


0: 640x640 1 platelets, 25 rbcs, 1 wbc, 1: 640x640 3 plateletss, 24 rbcs, 2 wbcs, 2: 640x640 1 platelets, 22 rbcs, 1 wbc, 3: 640x640 4 plateletss, 16 rbcs, 1 wbc, 4: 640x640 2 plateletss, 25 rbcs, 1 wbc, 5: 640x640 2 plateletss, 27 rbcs, 1 wbc, 6: 640x640 1 platelets, 17 rbcs, 1 wbc, 7: 640x640 20 rbcs, 1 wbc, 8: 640x640 1 platelets, 23 rbcs, 1 wbc, 9: 640x640 3 plateletss, 16 rbcs, 2 wbcs, 10: 640x640 2 plateletss, 22 rbcs, 1 wbc, 11: 640x640 1 platelets, 16 rbcs, 1 wbc, 12: 640x640 1 platelets, 27 rbcs, 1 wbc, 13: 640x640 20 rbcs, 1 wbc, 14: 640x640 2 plateletss, 25 rbcs, 1 wbc, 15: 640x640 14 rbcs, 1 wbc, 16: 640x640 22 rbcs, 1 wbc, 17: 640x640 18 rbcs, 1 wbc, 18: 640x640 2 plateletss, 18 rbcs, 1 wbc, 19: 640x640 2 plateletss, 29 rbcs, 2 wbcs, 20: 640x640 1 platelets, 19 rbcs, 1 wbc, 21: 640x640 2 plateletss, 30 rbcs, 1 wbc, 22: 640x640 3 plateletss, 18 rbcs, 1 wbc, 23: 640x640 18 rbcs, 1 wbc, 24: 640x640 21 rbcs, 1 wbc, 25: 640x640 2 plateletss, 22 rbcs, 1 wbc, 26: 640x640 1 plate